In [1]:
from sklearn.linear_model import SGDClassifier
import numpy as np
import collections
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from scipy.special import logsumexp
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch import Tensor
import geometry
import itertools
from collections import defaultdict
import collections
import target
import controler
import utils
from functools import reduce
import linear_regression
from multiprocessing import Pool
from functools import partial
import numpy as np
import geometry
import itertools 
from itertools import islice
import games
import feedexp3
import geometry_v2
import cpb
import feedexp3_v2
import bpm
import random_algo
import feedexp3_v3
import plotly.graph_objects as go
import TSPM

def evaluate_parallel(nbCores, n_folds, horizon, alg, game):
    print("nbCores:", nbCores, "nbFolds:", n_folds, "Horizon:", horizon)
    pool = Pool(processes = nbCores) 
    task = Evaluation(horizon)
    return np.asarray(  pool.map( partial( task.eval_policy_once, alg, game ), range(n_folds) ) ) 

def runif_in_simplex(n):
  ''' Return uniformly random vector in the n-simplex '''
  k = np.random.exponential(scale=1.0, size=n)
  return k / sum(k)

class Evaluation:

    def __init__(self, horizon ):
        self.horizon = horizon
        # self.outcome_distribution = outcome_distribution
    
    def get_outcomes(self, game, job_id):
        np.random.seed(job_id)
        # self.means = runif_in_simplex( self.game.n_outcomes )
        outcomes = np.random.choice( game.n_outcomes , p= list( game.outcome_dist.values() ), size= self.horizon) 
        return outcomes

    def get_feedback(self, game, action, outcome):
        return game.FeedbackMatrix[ action ][ outcome ]

    def eval_policy_once(self, alg, game, jobid):

        action_counter = np.zeros( (game.n_actions, self.horizon) )

        # generate outcomes obliviously
        outcomes = self.get_outcomes(game, jobid)

        for t in range(self.horizon):

            # policy chooses one action
            action = alg.get_action(t)

            # Environment chooses one outcome
            outcome = outcomes[t]

            print('t', t, 'action', action, 'outcome', outcome, )

            feedback =  self.get_feedback( game, action, outcome )


            alg.update(action, feedback, outcome)
            
            # print('nu', alg.nu / alg.n )

            for i in range(game.n_actions):
                if i == action:
                    action_counter[i][t] = action_counter[i][t-1] +1
                else:
                    action_counter[i][t] = action_counter[i][t-1]

        return action_counter

def eval_cpbvanilla_parallel(task, nbCores, n_folds, horizon, alpha):
    print("nbCores:", nbCores, "nbFolds:", n_folds, "Horizon:", horizon)
    pool = Pool(processes = nbCores) 
    return np.asarray(  pool.map( partial( task.cpb_vanilla_v2 ,alpha ), range(n_folds) ) ) 

def runif_in_simplex(n):
  ''' Return uniformly random vector in the n-simplex '''
  k = np.random.exponential(scale=1.0, size=n)
  return k / sum(k)


In [2]:
n_cores = 1
n_folds = 1
horizon = 100

# np.seterr(all='raise')

outcome_distribution =  {'spam':0.05,'ham':0.95}

# game = games.apple_tasting(False, outcome_distribution) 
game =  games.label_efficient(outcome_distribution) 
print('optimal action', game.i_star)
alg = TSPM.TSPM_alg(  game, horizon,)
task = Evaluation(horizon)

result = task.eval_policy_once(alg,game, 0)

# fig = go.Figure( )
# regret = np.array([ game.delta(i) for i in range(game.n_actions) ]).T @ np.mean(result,0) 
# xcoords = np.arange(0,horizon,1).tolist()

# fig.add_trace(go.Scatter(x=xcoords, y=regret, line=dict(color='blue'), mode='lines',  name='TPSM' )) # 


optimal action 1
n-actions 3 n-outcomes 2 alphabet 3
t 0 action 0 outcome 1
t 1 action 0 outcome 1
t 2 action 0 outcome 1
t 3 action 0 outcome 1
t 4 action 0 outcome 1
t 5 action 0 outcome 1
t 6 action 0 outcome 1
t 7 action 0 outcome 1
t 8 action 0 outcome 1
t 9 action 0 outcome 1
t 10 action 0 outcome 1
t 11 action 0 outcome 1
t 12 action 0 outcome 1
t 13 action 0 outcome 1
t 14 action 0 outcome 1
t 15 action 0 outcome 1
t 16 action 0 outcome 0
t 17 action 0 outcome 1
t 18 action 0 outcome 1
t 19 action 0 outcome 1
t 20 action 1 outcome 1
t 21 action 1 outcome 1
t 22 action 1 outcome 1
t 23 action 1 outcome 1
t 24 action 1 outcome 1
t 25 action 1 outcome 1
t 26 action 1 outcome 1
t 27 action 1 outcome 1
t 28 action 1 outcome 1
t 29 action 1 outcome 1
t 30 action 1 outcome 1
t 31 action 1 outcome 1
t 32 action 1 outcome 1
t 33 action 1 outcome 1
t 34 action 1 outcome 0
t 35 action 1 outcome 1
t 36 action 1 outcome 1
t 37 action 1 outcome 1
t 38 action 1 outcome 1
t 39 action 1 outcome

ValueError: matmul: Input operand 1 does not have enough dimensions (has 0, gufunc core with signature (n?,k),(k,m?)->(n?,m?) requires 1)

In [17]:
np.round(27 / 10 ) 

3.0

In [2]:
n_cores = 1
n_folds = 1
horizon = 2000

outcome_distribution =  {'spam':0.05,'ham':0.95}

game = games.apple_tasting(False, outcome_distribution)

algos = [ random_algo.Random(  game, horizon, ), 
           feedexp3.FeedExp3(  game, horizon, ),
           feedexp3_v3.FeedExp3(  game, horizon, ),
           cpb.CPB(  game, horizon,), TSPM.TSPM_alg(  game, horizon,), bpm.BPM(  game, horizon,  [0.5, 0.5 ], np.identity(2) )    ] 

colors = [ [0,0,0], [255,128,0], [0,153,0], [204,0,102], [29,196,74], [196,29,113]  ] 
labels = ['random', 'FeedExp3 (2001)', 'FeedExp3 (2006)', 'CPB',  'TSPM' , 'BPM-least' ] 

fig = go.Figure( )

for alg, color, label in zip( algos, colors, labels):

    r,g,b = color
    result = evaluate_parallel(n_cores, n_folds, horizon, alg, game)
    regret = np.array([ game.delta(i) for i in range(game.n_actions) ]).T @ np.mean(result,0) 
    xcoords = np.arange(0,horizon,1).tolist()
    std = np.array([ game.delta(i) for i in range(game.n_actions) ]).T @ np.std(result,0) 
    upper_regret = regret + std

    fig.add_trace(go.Scatter(x=xcoords, y=regret, line=dict(color='rgb({},{},{})'.format(r,g,b)), mode='lines',  name=label )) # 

    fig.add_trace(   go.Scatter( x=xcoords+xcoords[::-1], y=upper_regret.tolist()+regret.tolist()[::-1],  fill='toself', fillcolor='rgba({},{},{},0.2)'.format(r,g,b), 
                         line=dict(color='rgba(255,255,255,0)'),   hoverinfo="skip",  showlegend=False )

    )
    
fig.show(legend=True)
fig.update_yaxes(range=[0, 30] )
fig.show()


Restricted license - for non-production use only - expires 2023-10-25
n-actions 2 n-outcomes 2 alphabet 2
n-actions 2 n-outcomes 2 alphabet 2
nbCores: 1 nbFolds: 1 Horizon: 2000
nbCores: 1 nbFolds: 1 Horizon: 2000
nbCores: 1 nbFolds: 1 Horizon: 2000
nbCores: 1 nbFolds: 1 Horizon: 2000
nbCores: 1 nbFolds: 1 Horizon: 2000
limit trial exceeded


ValueError: matmul: Input operand 1 does not have enough dimensions (has 0, gufunc core with signature (n?,k),(k,m?)->(n?,m?) requires 1)

In [14]:
n_cores = 16
n_folds = 100
horizon = 2000

outcome_distribution =  {'spam':0.05,'ham':0.95}

game = games.label_efficient(outcome_distribution)

algos = [ random_algo.Random(  game, horizon, ), 
           feedexp3.FeedExp3(  game, horizon, ),
           feedexp3_v3.FeedExp3(  game, horizon, ),
           cpb.CPB(  game, horizon,),  TSPM.TSPM_alg(  game, horizon,), bpm.BPM(  game, horizon,  [0.5, 0.5 ], np.identity(2) )    ] 

colors = [ [0,0,0], [255,128,0], [0,153,0], [204,0,102], [29,196,74], [196,29,113]  ] 
labels = ['random', 'FeedExp3 (2001)', 'FeedExp3 (2006)', 'CPB',  'TSPM' , 'BPM-least' ] 


fig = go.Figure( )

for alg, color, label in zip( algos, colors, labels):

    r,g,b = color
    result = evaluate_parallel(n_cores, n_folds, horizon, alg, game)
    regret = np.array([ game.delta(i) for i in range(game.n_actions) ]).T @ np.mean(result,0) 
    xcoords = np.arange(0,horizon,1).tolist()
    std = np.array([ game.delta(i) for i in range(game.n_actions) ]).T @ np.std(result,0) 
    upper_regret = regret + std

    fig.add_trace(go.Scatter(x=xcoords, y=regret, line=dict(color='rgb({},{},{})'.format(r,g,b)), mode='lines',  name=label )) # 

    fig.add_trace(   go.Scatter( x=xcoords+xcoords[::-1], y=upper_regret.tolist()+regret.tolist()[::-1],  fill='toself', fillcolor='rgba({},{},{},0.2)'.format(r,g,b), 
                         line=dict(color='rgba(255,255,255,0)'),   hoverinfo="skip",  showlegend=False )

    )
    
fig.show(legend=True)
fig.update_yaxes(range=[0, 30] )
fig.show()


nbCores: 16 nbFolds: 100 Horizon: 2000
nbCores: 16 nbFolds: 100 Horizon: 2000
nbCores: 16 nbFolds: 100 Horizon: 2000
nbCores: 16 nbFolds: 100 Horizon: 2000
